In [2]:
import numpy as np
import sys
# option to import from github folder
#sys.path.insert(0, '/media/carsen/DATA2/Github/suite2p')
sys.path.insert(0, 'C:/Users/carse/github/suite2p')

import imp
from suite2p import run_s2p
from suite2p import utils
from skimage.external.tifffile import imread
from ScanImageTiffReader import ScanImageTiffReader
import json

with open('/home/carsen/public/for_Carsen/trial/ops.json', 'r') as f:
    ops = json.load(f)
ops0 = run_s2p.default_ops()
ops = {**ops0, **ops}
ops['data_path'] = '/home/carsen/public/for_Carsen/trial/'

In [6]:
ops

{'look_one_level_down': False,
 'fast_disk': [],
 'delete_bin': 1,
 'mesoscan': 1,
 'h5py': [],
 'h5py_key': 'data',
 'save_path0': 'G:/Teena/Teena/Joys5/040919',
 'subfolders': [],
 'nplanes': 3,
 'nchannels': 1,
 'functional_chan': 1,
 'diameter': [6, 9],
 'tau': 2,
 'fs': 44.6860562406527,
 'save_mat': False,
 'combined': 1,
 'num_workers': 0,
 'num_workers_roi': 4,
 'do_registration': True,
 'keep_movie_raw': 0,
 'nimg_init': 400,
 'batch_size': 1000,
 'maxregshift': 0.1,
 'align_by_chan': 1,
 'reg_tif': False,
 'reg_tif_chan2': False,
 'subpixel': 10,
 'do_phasecorr': True,
 'smooth_sigma': 1.15,
 'nonrigid': 1,
 'block_size': [128, 128],
 'snr_thresh': 1.2,
 'maxregshiftNR': 5,
 'connected': True,
 'navg_frames_svd': 5000,
 'nsvd_for_roi': 1000,
 'max_iterations': 20,
 'smooth_masks': 1,
 'threshold_scaling': 1.0,
 'max_overlap': 0.75,
 'ratio_neuropil': 6.0,
 'ratio_neuropil_to_cell': 3,
 'tile_factor': 1.0,
 'inner_neuropil_radius': 2,
 'outer_neuropil_radius': inf,
 'min_neuro

In [2]:
print(ops['data_path'])
# copy ops to list where each element is ops for each plane
# load json file with line start stops
if 'lines' not in ops:
    fpath = os.path.join(ops['data_path'][0], '*json')
    fs = glob.glob(fpath)
    with open(fs[0], 'r') as f:
        opsj = json.load(f)
    ops['nplanes'] = len(opsj)
else:
    ops['nplanes'] = len(ops['lines'])
ops1 = init_ops(ops)
ops = ops1[0]

if 'lines' not in ops:
    for j in range(len(ops1)):
        ops1[j] = {**ops1[j], **opsj[j]}.copy()

# open all binary files for writing
# look for tiffs in all requested folders
ops1, fs, reg_file, reg_file_chan2 = find_files_open_binaries(ops1, False)

nplanes = ops1[0]['nplanes']
nchannels = ops1[0]['nchannels']
if nchannels>1:
    nfunc = ops['functional_chan']-1
else:
    nfunc = 0
batch_size = ops['batch_size']

# which tiff reader works for user's tiffs
sktiff = choose_tiff_reader(fs[0], ops1[0])

# loop over all tiffs
which_folder = -1
for ik, file in enumerate(fs):
    # open tiff
    tif, Ltif = open_tiff(file, sktiff)
    if ops['first_tiffs'][ik]:
        which_folder += 1
    ix = 0
    while 1:
        if ix >= Ltif:
            break
        nfr = min(Ltif - ix, batch_size)
        if sktiff:
            im = imread(file, pages = range(ix, ix + nfr), fastij = False)
        else:
            if Ltif==1:
                im = tif.data()
            else:
                im = tif.data(beg=ix, end=ix+nfr)
        if im.size==0:
            break
        #im = io.imread(file)
        if len(im.shape)<3:
            im = np.expand_dims(im, axis=0)
        nframes = im.shape[0]
        for j in range(0,nplanes):
            if ik==0:
                ops1[j]['meanImg'] = np.zeros((len(ops1[j]['lines']),im.shape[2]),np.float32)
                if nchannels>1:
                    ops1[j]['meanImg_chan2'] = np.zeros((len(ops1[j]['lines']),im.shape[2]),np.float32)
                ops1[j]['nframes'] = 0
            imj = im[:,ops1[j]['lines'],:].astype(np.int16)
            i0 = nchannels
            if nchannels>1:
                nfunc = ops['functional_chan'] - 1
            else:
                nfunc = 0
            im2write = imj[np.arange(int(i0)+nfunc, nframes, nchannels, int), :, :]
            ops1[j]['meanImg'] += im2write.astype(np.float32).sum(axis=0)
            reg_file[j].write(bytearray(im2write))
            if nchannels>1:
                im2write = imj[np.arange(int(i0)+1-nfunc, nframes, nchannels, int), :, :]
                reg_file_chan2[j].write(bytearray(im2write))
                ops1[j]['meanImg_chan2'] += im2write.astype(np.float32).sum(axis=0)
            ops1[j]['nframes'] += im2write.shape[0]
            ops1[j]['frames_per_folder'][which_folder] += im2write.shape[0]
        ix+=nframes

In [6]:
im.shape

(100, 768, 768)

In [ ]:
data = imread(fs[0])
#im=tif.data()
print(data.shape)

In [ ]:
data = data / 2

In [ ]:
import matplotlib.pyplot as plt
from suite2p import register, nonrigid, utils
import imp
imp.reload(register)
imp.reload(nonrigid)

ops['Ly'] = data.shape[1]
ops['Lx'] = data.shape[2]

#refImg=register.pick_init_init(ops, data[::4,:,:])
#refImg =register.refine_init_init(ops, data[::4,:,:], refImg)
#plt.imshow(refImg)
#plt.show()
print('mean img')

dwrite = data[:200,:,:].copy()

#for j in range(2):
#    if j>0:
#        refImg=register.pick_init_init(ops, dwrite[:,:,:])
#        refImg =register.refine_init_init(ops, dwrite[:,:,:], refImg)
#        print('mean img')
    
maskMul, maskOffset, cfRefImg = register.prepare_masks(refImg.copy(), ops)
if ops['nonrigid']:
    ops = utils.make_blocks(ops)
    maskMulNR, maskOffsetNR, cfRefImgNR = nonrigid.prepare_masks(refImg, ops)
    refAndMasks = [maskMul, maskOffset, cfRefImg, maskMulNR, maskOffsetNR, cfRefImgNR]
else:
    refAndMasks = [maskMul, maskOffset, cfRefImg]
dwrite, ymax, xmax, cmax, yxnr = register.phasecorr(dwrite, refAndMasks, ops)
print('registered')

    

In [ ]:
type(data[0,0,0])

In [ ]:
plt.imshow(dwrite.mean(axis=0))

In [ ]:
maskMul, maskOffset, cfRefImg = register.prepare_masks(refImg.copy(), ops)
plt.imshow(np.real(np.fft.ifft2(np.conj(cfRefImg[0,:,:]))))

In [ ]:
imp.reload(register)



plt.figure(figsize=(10,10))
plt.imshow(np.real(fft.ifft2(np.conj(cfRefImg))))
#plt.imshow(fhg)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(refImg)

In [ ]:
plt.imshow(data[1,:,:])

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(ymax[:])
plt.plot(xmax[:])
plt.show()

In [ ]:
plt.plot(yxnr[0][-1,:])
plt.show()
yxnr[0].shape

In [ ]:
import numpy as np

from matplotlib import animation, rc
from IPython.display import HTML
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots(figsize=(10,10))

#dwrite = data

#line, = ax.plot([], [], lw=2)
im = ax.imshow(dwrite[0,:,:],cmap='gray')

# initialization function: plot the background of each frame
def init():
    im.set_data(dwrite[0,:,:])#,cmap='gray')
    return (im,)
# animation function. This is called sequentially
def animate(i):
    im.set_data(dwrite[i,:,:])#,cmap='gray')
    return (im,)
# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=100, blit=True)
HTML(anim.to_html5_video())

In [ ]:
dwrite.max()

In [ ]:
2**15

In [ ]:
plt.figure(figsize=(20,10))
ix=186
vm=-40
vx=40
plt.subplot(1,2,1)
plt.imshow(data[ix,:,:]-data[ix+1,:,:],vmin=vm,vmax=vx)
plt.subplot(1,2,2)
plt.imshow(dout[ix,:,:]-dout[ix+1,:,:],vmin=vm,vmax=vx)

In [ ]:
dout=data

In [ ]:
from skimage.external.tifffile import TiffFile

#fs = ['/home/carsen/TIFFS/TSeries-20170508-D43-001_Cycle00001_Ch2_000001.ome.tif']
fs = ['/home/carsen/TIFFS/MP032/2017-08-01_6_M170714_MP032_2P_001_001.tif'];

import time

tic = time.time()
tif = TiffFile(fs[0])
print(time.time()-tic)

print(tif[0].asarray())

In [ ]:
1800/12 * 5

In [ ]:
F = np.load('/home/carsen/Downloads/suite2p/plane0/F.npy')
F.shape

In [ ]:
import scipy.io

Fall = scipy.io.loadmat('/home/carsen/Downloads/suite2p/plane0/Fall.mat',squeeze_me=True)

In [ ]:
Fall['F'].shape